In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
import sys
import os
import pandas as pd
import itertools

# Adicionar diretório src ao path
project_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_path)

from src.ga_runner import run_ga
from src.individual import create_individual
from src.fitness import evaluate_fitness
from src.selection import roulette_selection, ranking_selection, tournament_selection
from src.crossover import group_based_crossover, greedy_table_merge_crossover
from src.mutation import swap_mutation, one_point_mutation, multiple_point_mutation
from copy import deepcopy

In [ ]:
selection_types = ["tournament", "ranking", "roulette"]
crossover_types = ["group", "merge"]
mutation_types = ["swap", "one_point", "multiple_point"]
elitism_options = [True, False]

results = []

for sel, cross, mut, elit in itertools.product(selection_types, crossover_types, mutation_types, elitism_options):
    print("\n")
    print(f"Running: {sel} - {cross} - {mut} - elitism={elit}")
    best_solution, best_score, best_fitness_per_gen = run_ga(
        pop_size=100,
        generations=100,
        elite_size=2,
        use_elitism=elit,
        selection_type=sel,
        crossover_type=cross,
        mutation_type=mut,
        mutation_prob=0.2,
        seed=42
    )
    results.append({
        "selection": sel,
        "crossover": cross,
        "mutation": mut,
        "elitism": elit,
        "history": best_fitness_per_gen,
        "final_score": best_score
    })




Running: tournament - group - swap - elitism=True
Geração 0 | Best fitness: 19752.00
Geração 10 | Best fitness: 19752.00
Geração 20 | Best fitness: 20553.00
Geração 30 | Best fitness: 22461.00
Geração 40 | Best fitness: 23392.00
Geração 50 | Best fitness: 23392.00
Geração 60 | Best fitness: 23661.00
Geração 70 | Best fitness: 25692.00
Geração 80 | Best fitness: 27902.00
Geração 90 | Best fitness: 27902.00


Running: tournament - group - swap - elitism=False
Geração 0 | Best fitness: 19752.00
Geração 10 | Best fitness: 16400.00
Geração 20 | Best fitness: 13800.00
Geração 30 | Best fitness: 11500.00
Geração 40 | Best fitness: 11500.00
Geração 50 | Best fitness: 11500.00
Geração 60 | Best fitness: 11500.00
Geração 70 | Best fitness: 11500.00
Geração 80 | Best fitness: 11500.00
Geração 90 | Best fitness: 11500.00


Running: tournament - group - one_point - elitism=True
Geração 0 | Best fitness: 19752.00


In [ ]:
plt.figure(figsize=(14, 8))
for r in results:
    label = f"{r['selection']}-{r['crossover']}-{r['mutation']} | elit={r['elitism']}"
    plt.plot(r["history"], label=label)
plt.xlabel("Geração")
plt.ylabel("Melhor Fitness")
plt.title("Evolução do Fitness por Geração")
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
summary = pd.DataFrame([{
    "Selection": r["selection"],
    "Crossover": r["crossover"],
    "Mutation": r["mutation"],
    "Elitism": r["elitism"],
    "Final Score": r["final_score"]
} for r in results])

summary = summary.sort_values(by="Final Score", ascending=False)
summary.reset_index(drop=True, inplace=True)
summary
